In [1]:
import time
import numpy as np

In [2]:
def load_dataset(data_dir=''):
    """Load the train and test examples 
    """
    x_train = np.load("data/x_train.npy")
    y_train = np.load("data/y_train.npy")
    x_test = np.load("data/x_test.npy")
    y_test = np.load("data/y_test.npy")

    return x_train, y_train, x_test, y_test

In [3]:
x_train, y_train, x_test, y_test = load_dataset()

In [4]:
x_train.shape

(50000, 784)

In [5]:
x_sub = x_train[0:]
y_sub = y_train[0:]

In [6]:
def one_hotter(a, colors=256):
    #print(colors)
    b = np.zeros((a.size, colors))
    b[np.arange(a.size),a] = 1
    return b.T

In [7]:
def train(x_train, y_train):
    start = time.time()
    ans = np.zeros((10,256,784))
    for i, x in enumerate(x_train):
        y = y_train[i]
        hot = one_hotter(x)
        ans[y]+=hot
    print(time.time()-start)
    return ans

In [8]:
trained = train(x_sub, y_sub)

69.25726366043091


In [9]:
#sum(np.apply_along_axis(one_hotter, -1, x_sub))

In [10]:
classes, counts = np.unique(y_sub, return_counts=True)

In [11]:
p_class = np.log(counts/sum(counts))

In [12]:
print(p_class)

[-2.30258509 -2.30258509 -2.30258509 -2.30258509 -2.30258509 -2.30258509
 -2.30258509 -2.30258509 -2.30258509 -2.30258509]


In [19]:
def p(c, pixel, val):
    count = trained[c][val][pixel]
    total = counts[c]
    return np.log((count + 1)/(total+1))

In [20]:
def p_total(c, x):
    #print(c, x.shape)
    total = sum([p(c, pixel, col) for pixel, col in enumerate(x)])
    return total + p_class[c]

In [21]:
start = time.time()
correct, all_samples = 0,0
for i, x in enumerate(x_test[:10]):
    vals = []
    for c in classes:
        #total = sum([p(pixel, col, c) for pixel, col in enumerate(x)])
        #vals.append(total+p_class[c])
        
        vals.append(p_total(c, x))
    if np.argmax(vals) == y_test[i]:
        correct+=1
    all_samples+=1
print(correct, all_samples)
print(time.time()-start)

8 10
0.20096302032470703


In [22]:
def hi(c, x):
    print(c, x.shape)

In [23]:
from multiprocessing.dummy import Pool as ThreadPool
pool = ThreadPool(8)

In [27]:
start = time.time()
correct, all_samples = 0,0
classes = np.array(classes)
print(classes)
for i, x in enumerate(x_test[:]):
    vals = list(pool.map(lambda c: p_total(c, x), classes))
    #vals = np.apply_along_axis(hi, 1, classes, x)
    if np.argmax(vals) == y_test[i]:
        correct+=1
    all_samples+=1
print(correct, all_samples)
print(time.time()-start)

[0 1 2 3 4 5 6 7 8 9]
7421 10000
203.29499077796936


In [18]:
trained[0][0][:].shape

(784,)